In [1]:
import pandas as pd
k2 = pd.read_pickle('data\\k2\\k2_data_treated.pkl')
k2_classification = k2[k2['classification'].isin(['planet', 'not planet'])]
x_k2 = k2_classification.drop(['classification', 'mission'], axis=1)  # Features
y_k2 = k2_classification['classification']  # Rótulos

from sklearn.model_selection import train_test_split

x_k2_treino, x_k2_teste, y_k2_treino, y_k2_teste = train_test_split(x_k2, y_k2, test_size=0.2, random_state=42)

In [2]:
import numpy as np

# Base treino não balanceada. Proporção
unique_values, counts = np.unique(y_k2_treino, return_counts=True)
prop = counts/y_k2_treino.size
print(unique_values,counts,prop)

['not planet' 'planet'] [ 245 1677] [0.12747138 0.87252862]


In [3]:
### montando base de treino balanceada

from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42)
x_k2_treino_bal, y_k2_treino_bal = smt.fit_resample(x_k2_treino, y_k2_treino)

In [4]:
# base de treino balanceada. Proporção
unique_values, counts = np.unique(y_k2_treino_bal, return_counts=True)
prop = counts/y_k2_treino_bal.size
print(unique_values,counts,prop)

['not planet' 'planet'] [1657 1657] [0.5 0.5]


In [8]:
from sklearn.naive_bayes import GaussianNB
### treinando modelo com dados nao balanceados e gerando previsoes com o modelo nao balanceado
modeloNB_nao_balanceado = GaussianNB()
modeloNB_nao_balanceado.fit(x_k2_treino,y_k2_treino)
y_k2_previsto_nao_balanceado = modeloNB_nao_balanceado.predict(x_k2_teste)

### treinando modelo com dados balanceados e gerando previsões com modelo balanceado
modeloNB_balanceado = GaussianNB()
modeloNB_balanceado.fit(x_k2_treino_bal,y_k2_treino_bal)
y_k2_previsto_balanceado = modeloNB_balanceado.predict(x_k2_teste)

In [18]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [19]:
acc_nao_balanceado = accuracy_score(y_k2_teste,y_k2_previsto_nao_balanceado)
pre_nao_balanceado = precision_score(y_k2_teste,y_k2_previsto_nao_balanceado,pos_label='planet') 
rec_nao_balanceado = recall_score(y_k2_teste,y_k2_previsto_nao_balanceado,pos_label='planet')
f1_nao_balanceado = f1_score(y_k2_teste,y_k2_previsto_nao_balanceado,pos_label='planet')

acc_balanceado = accuracy_score(y_k2_teste,y_k2_previsto_balanceado)
pre_balanceado = precision_score(y_k2_teste,y_k2_previsto_balanceado,pos_label='planet')
rec_balanceado = recall_score(y_k2_teste,y_k2_previsto_balanceado,pos_label='planet')
f1_balanceado = f1_score(y_k2_teste,y_k2_previsto_balanceado,pos_label='planet')

### obs.: como nosso target não é 1 e 0, temos que informar às métricas precision, recall e f1_score quem é o target positivo. Por isso o argumento pos_label = 'planet'

In [20]:
acc_nao_balanceado, acc_balanceado

(0.8731808731808732, 0.8378378378378378)

In [21]:
pre_nao_balanceado,pre_balanceado

(0.919431279620853, 0.9399477806788512)

In [22]:
rec_nao_balanceado,rec_balanceado

(0.9349397590361446, 0.8674698795180723)

In [23]:
f1_nao_balanceado,f1_balanceado

(0.927120669056153, 0.9022556390977443)

In [86]:
### tratamento de outliers
### é necessário transformar o target em númeors, pois a tecnica de identificação de outliers IsolationForest só trabalha com números
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_k2_treino) 
### juntar dados de treinos
dados_para_analisar_outliers = np.column_stack((x_k2_treino, y_encoded))  
### identificar os outliers (supondo 5% de contaminação na base (ajustável))
from sklearn.ensemble import IsolationForest
clf = IsolationForest(contamination=0.01)
labels = clf.fit_predict(dados_para_analisar_outliers)
### rearrumando a base removendo outliers
dados_filtrados = dados_para_analisar_outliers[labels == 1]
x_k2_treino_filtrado = dados_filtrados[:, :-1]
y_k2_treino_filtrado = dados_filtrados[:, 1]
### Utilizamos o mesmo objeto de encode para retransformar os target numéricos em categoricos
y_k2_treino_filtrado = encoder.inverse_transform(y_k2_treino_filtrado.astype(int))

In [87]:
x_k2_treino_filtrado

array([[3.08639068e-02, 6.38509162e-01, 5.27648435e-02, ...,
        6.73032664e-05, 1.47972504e-02, 1.36198262e-02],
       [4.46870369e-01, 7.34932325e-01, 5.31963057e-02, ...,
        3.88865338e-05, 1.51470803e-02, 4.17581781e-03],
       [3.45518923e-01, 8.17389233e-01, 7.27933720e-02, ...,
        1.60723228e-05, 2.39887079e-02, 2.27063666e-03],
       ...,
       [1.17015825e-02, 5.01283318e-01, 6.33676155e-02, ...,
        1.13492305e-04, 5.73701059e-03, 4.17358306e-04],
       [5.09186138e-01, 4.13090540e-01, 7.93869975e-02, ...,
        4.56402006e-05, 1.51506057e-02, 2.04864211e-02],
       [4.77645256e-01, 5.99179149e-01, 5.02988048e-02, ...,
        2.25492182e-04, 1.34146556e-03, 2.23338372e-03]])

In [88]:
### treinando modelo com dados com outliers removidos
modeloNB_limpo = GaussianNB()
modeloNB_limpo.fit(x_k2_treino_filtrado, y_k2_treino_filtrado)
y_k2_previsto_limpo = modeloNB_limpo.predict(x_k2_teste.to_numpy())

In [89]:
valores_unicos, contagens = np.unique(y_k2_treino_filtrado, return_counts=True)
valores_unicos, contagens

(array(['not planet'], dtype=object), array([1902], dtype=int64))

In [85]:
acc_limpo = accuracy_score(y_k2_teste,y_k2_previsto_limpo)
pre_limpo = precision_score(y_k2_teste,y_k2_previsto_limpo,pos_label='planet')
rec_limpo = recall_score(y_k2_teste,y_k2_previsto_limpo,pos_label='planet')
f1_limpo = f1_score(y_k2_teste,y_k2_previsto_limpo,pos_label='planet')

d:\Drive Unesc\Meu Drive\2024-1\TCC Janaína\Projeto3\EstelarClass\venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [62]:
acc_nao_balanceado,acc_limpo

(0.8731808731808732, 0.13721413721413722)

In [58]:
pre_nao_balanceado,pre_limpo

(0.919431279620853, 0.0)

In [59]:
rec_nao_balanceado,rec_limpo

(0.9349397590361446, 0.0)

In [60]:
f1_nao_balanceado,f1_limpo

(0.927120669056153, 0.0)